In [1]:
from itertools import chain

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm


import torch
import spacy
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

/Users/elnardu/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# initialize hyper hyper parameters
torch.manual_seed(420)
np.random.seed(420)

-------------------

# Load data

In [3]:
fallacies_df = pd.read_csv('../data/fallacies.csv', index_col=0)
approved_df = pd.read_csv('../data/approved.csv', index_col=0)

In [4]:
df = pd.concat([
    fallacies_df,
    approved_df[approved_df.n_supporters >= 5].drop("n_supporters", axis=1)
])
df.fallacy_reason = df.fallacy_reason.fillna('')
df = df[~df.premise_content.isna()]

In [5]:
vc = df.fallacy_type.value_counts()

In [6]:
df = df[df.fallacy_type.isin(vc.head(10).index)]

# Text preprocessing

In [7]:
nlp = spacy.load('en_core_web_sm')

In [8]:
def preprocess_sentence(sent):
    sent = sent.lower()
    sent = nlp(sent)
    words = map(lambda x: x.text, sent)
    return list(words)

In [9]:
df['premise_content_preprocessed'] = df.premise_content.apply(preprocess_sentence)

In [10]:
df = df.sample(frac=1).reset_index(drop=True) # shuffle the data

In [11]:
train_df, test_df = train_test_split(df, test_size=0.1)

In [12]:
word_vocab = {"<oov>", "<pad>"}
word_vocab = word_vocab.union(
    set(chain.from_iterable(map(lambda x: x[1]["premise_content_preprocessed"], train_df.iterrows())))
)
word_to_ix = {word: i for i, word in enumerate(word_vocab)}

In [13]:
fallacy_vocab = set(df.fallacy_type.unique())
fallacy_to_ix = {word: i for i, word in enumerate(fallacy_vocab)}

# Neural Net

In [14]:
class Net(nn.Module):
    def __init__(self, word_vocab_size, word_embedding_dim, fallacy_vocab_size, max_sent_size):
        super(Net, self).__init__()
        
        self.word_embeddings = nn.Embedding(
            word_vocab_size, word_embedding_dim
        )  # random init

        hidden = 100
        self.fc1 = nn.Linear(word_embedding_dim, hidden)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden * max_sent_size, fallacy_vocab_size)
        
    def forward(self, word_inputs):
        word_embeds = self.word_embeddings(word_inputs)
        h1 = self.fc1(word_embeds)
        a1 = self.relu(h1)
        h2 = self.fc2(a1.view(a1.shape[0], -1))
        return h2

In [15]:
MAX_SENT_SIZE = 100

In [16]:
def pad_and_convert_to_ints(data):
    X = np.full((len(data), MAX_SENT_SIZE), word_to_ix["<pad>"])

    for i, (_, x) in enumerate(data.iterrows()):
        X[i, :len(x["premise_content_preprocessed"])] = [
            (word_to_ix[word] if word in word_to_ix else word_to_ix['<oov>'])
            for word in x["premise_content_preprocessed"]
        ]

    return X

In [17]:
trainX = pad_and_convert_to_ints(train_df)
testX = pad_and_convert_to_ints(test_df)

In [18]:
trainY = train_df.fallacy_type.apply(lambda x: fallacy_to_ix[x]).values
testY = test_df.fallacy_type.apply(lambda x: fallacy_to_ix[x]).values

In [19]:
net = Net(
    len(word_vocab),
    100,
    len(fallacy_vocab),
    MAX_SENT_SIZE,
)
opt = optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999))
criterion = nn.CrossEntropyLoss()

In [20]:
def train_epoch(X, Y, model, opt, criterion, batch_size=50):
    model.train()
    losses = []
    for beg_i in range(0, X.shape[0], batch_size):
        x_batch = X[beg_i : beg_i + batch_size, :]
        y_batch = Y[beg_i : beg_i + batch_size]
        x_batch = torch.tensor(x_batch)
        y_batch = torch.tensor(y_batch)

        opt.zero_grad()

        y_pred = model(x_batch)

        loss = criterion(y_pred, y_batch)

        loss.backward()
        
        opt.step()

        losses.append(loss.data.numpy())

    return [sum(losses) / float(len(losses))]

In [21]:
e_losses = []
num_epochs = 50
for e in tqdm(range(num_epochs)):
    e_losses += train_epoch(trainX, trainY, net, opt, criterion, batch_size=100)

100%|██████████| 50/50 [00:49<00:00,  1.01it/s]


In [22]:
e_losses

[1.648753839380601,
 1.21101713180542,
 0.9515892758088953,
 0.7630115361774669,
 0.6125645865412319,
 0.4951530668665381,
 0.408632590490229,
 0.3428669269470608,
 0.2935536319718641,
 0.2552408746936742,
 0.22508155182003975,
 0.20155408912721803,
 0.1859373873209252,
 0.16836540346198223,
 0.15941342382746584,
 0.14679257144384525,
 0.1432666515602785,
 0.13273896019467535,
 0.1333230619921404,
 0.12349908179877435,
 0.12633169113713152,
 0.11653816256233875,
 0.12042990858283113,
 0.11153627923854134,
 0.11452598494532354,
 0.10918897406800705,
 0.10946088349994491,
 0.10514081118847518,
 0.10541462008019581,
 0.10281893854741664,
 0.10362636391073465,
 0.10126961147248306,
 0.10245951656799983,
 0.1000685046070858,
 0.10098900229615324,
 0.09892782963374082,
 0.10043832237887032,
 0.0977024675675613,
 0.09950917482595234,
 0.0964931370690465,
 0.09885691539110507,
 0.09524452180930358,
 0.09799717498176239,
 0.09421895990860374,
 0.09737113296218655,
 0.09327493162880487,
 0.09671

# Metrics

In [23]:
with torch.no_grad():
    net.eval()
    x = torch.tensor(testX)
    y_pred = net(x)

In [24]:
score = f1_score(testY, y_pred.numpy().argmax(axis=1), average='micro')

In [25]:
score

0.6611111111111111